# 1

## Задание №1:
1. Скачать минимум 100 текстовых страниц с помощью краулера.
2. Записать каждую страницу в отдельный текстовый файл.
3. Создать файл index.txt в котором хранится номер документа и ссылка на страницу.
Входным аргументом программы должен быть веб-адрес страницы.
Ссылки с первой страницы ведут на другие, которые также скачиваются и заносятся в файл index.txt. Если на первой странице не набралось достаточного количества страниц (100), то операция повторяется для дочерних страниц первой.
Каждая страница должна содержать не менее 1000 слов.


In [ ]:
import sys
import os, glob
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import requests
import html2text

def getDomain(url):
    domain='https://'+ urlparse(url).hostname
    return domain

def createPageTextFile(fileName, text):
    global folder
    file = open("{0}\{1}.txt".format(folder,fileName),"x",encoding="utf-8")
    file.write(text)
    file.close()
    
def getPageUrls(url):
    page=requests.get(url)
    soup=BeautifulSoup(page.text,"html.parser")
    domain=getDomain(url)
    items=soup.find_all('a')
    urls=[]
    
    for item in items:
        if item is None:
            continue
        url=item.get('href')
        if url is None:
            continue    
        if url.startswith('/'):
            urls.append(domain+url)
        elif url.startswith('http'):
            urls.append(url) 
            
    return list(dict.fromkeys(urls))

def getPageText(soup):
    h=html2text.HTML2Text()
    h.ignore_link=True
    h.ignore_image=True
    h.escape_snob=True
    h.skip_internal_link=True
    return h.handle(soup.prettify())

def pageScraping(url,file_ind,url_ind):
    global childPageUrls,f
    urls=getPageUrls(url)
##############################################
    for element in urls:
        if file_ind>100:
            break
        page=requests.get(element)
        soup=BeautifulSoup(page.text,"html.parser")
        text=getPageText(soup)
        if len(text.split())<1000:
            continue
        createPageTextFile(file_ind,text)
        f.write('{0} {1}\n'.format(file_ind,element))
        file_ind+=1
        
    if file_ind<=100:
        pageScraping(childPageUrls[url_ind+1],file_ind,url_ind+1)
        
        
        
try:
    site="https://www.microsoft.com/ru-ru"
    #site="https://www.litres.ru"
    childPageUrls=getPageUrls(site)
    
    folder=r'...\IS\Folder'
    files=glob.glob(folder+'\*')
    for f in files:
        os.remove(f)
    f=open(r'...\IS\index.txt','w',encoding="utf-8")
    pageScraping(site,1,-1)
    f.close()
    print('Done!')
except Exception as error:
    print(error)
    
    #другой

## Задание 2

1. Из сохраненных документов выделить отдельные слова (токенизация).
2. Лемматизировать токены (допускается использование сторонних библиотек, устно могу спросить какие есть способы для лемматизации).
Русский язык обязателен.

In [13]:
import re
import os, glob
import pymorphy2
from nltk.tokenize import word_tokenize
import nltk

morph=pymorphy2.MorphAnalyzer()

filesFolder=r'...\IS\Folder'
lemmasFolder=r'...\IS\Lemmas'

stop=['-','--','и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'чтоб', 'без', 'будто', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между']

try:
    files=glob.glob(lemmasFolder+'\*')
    for f in files:
        os.remove(f)
    for i in range(1,101):
        f=open('{0}\{1}.txt'.format(filesFolder,i),"r",encoding="utf-8")
        text=f.read()
        text=re.sub(r"[^a-zA-Zа-яёА-ЯЁ0-9-]+",' ',text)
        words=word_tokenize(text.lower())
        words=[word for word in words if word not in stop]
        terms=[morph.parse(word)[0].normal_form for word in words]
        file=open("{0}\{1}.txt".format(lemmasFolder,i),"x",encoding="utf-8")
        file.write('\n'.join(terms))
        file.close()
    print('Done!')
    
except Exception as error:
    print(error)

Done!


# Задание 3

1. Создать инвертированный список терминов (индекс).
2. Реализовать булев поиск по построенному индексу (т.е. вводится выражение содержащее слова с тремя логическими И, ИЛИ, НЕ, по которому выдается список документов, содержащий данное выражение).

Примеры:
str1 & str2 | str3    str1 & !str2 | !str3
str1 | str2 | str3    str1 | !str2 | !str3

In [14]:
folder=r'...\IS'
lemmasFolder=r'...\IS\Lemmas'

def createIndex(lemmas,folder):
    indf=open('{0}\windex.txt'.format(folder),"w",encoding='utf-8')
    
    
    key=lemmas[0].split()[0]
    docs=[lemmas[0].split()[1]]
    for i in range(1,len(lemmas)):
        elem=lemmas[i].split()
        if elem[0]==key:
            docs.append(elem[1])
        else:
            indf.write('{0} {1}\n'.format(key,','.join(docs)))
            key=elem[0]
            docs=[elem[1]]
    indf.write('{0} {1}\n'.format(key,','.join(docs)))
    indf.close()
    
def createLemmasDct(lemmasFolder):
    lemmas=[]
    for i in range(1,101):
        f=open('{0}\{1}.txt'.format(lemmasFolder,i),'r',encoding='utf-8')
        lines=f.readlines()
        for line in lines:
            lemmas.append('{0} {1}'.format(line,i))
    lemmas=list(dict.fromkeys(lemmas))
    lemmas=sorted(lemmas)
    return lemmas

try:
    lemmas=createLemmasDct(lemmasFolder)
    createIndex(lemmas,folder)
    print('Done!')
except Exception as error:
    print(error)

Done!


In [39]:
import pymorphy2

def splitDocsIndex(word,dct):
    if word in dct.keys():
        return dct[word].split(',')
    return []

def lemmatize(words):
    morph=pymorphy2.MorphAnalyzer()
    terms=[morph.parse(word)[0].normal_form for word in words]
    return terms

def getInd(i):
    return int(i/2)

def mergeLists(list1,list2):
    return [doc for doc in list1 if doc in list2]

def getTermDict():
    global folder
    f=open('{0}\windex.txt'.format(folder),"r",encoding='utf-8')
    lines=f.readlines()
    termDict={line.split()[0]: line.split()[1] for line in lines}
    return termDict

def getPhraseDocs(phrase,termDict):
    global not_s
    phraseDocs=[]
    for i in range(0,5,2):
        if phrase[i][0]==not_s:
            wordDoc=splitDocsIndex(phrase[i].split(not_s)[1],termDict)
            phraseDocs.append([doc for doc in allDocs if doc not in wordDoc])
        else:
            phraseDocs.append(splitDocsIndex(phrase[i],termDict))
    return phraseDocs

def mergePhraseDocs(phrase,phraseDocs):
    global and_s
    if and_s in phrase:
        ind=phrase.index(and_s)
        list1=phraseDocs[getInd(ind-1)]
        list2=phraseDocs[getInd(ind+1)]
        mergedList=mergeLists(list1,list2)
        if ind==1:
            nextInd=3
            reqInd=4
        else:
            nextInd=1
            reqInd=0
        if phrase[nextInd]==and_s:
            resultList=mergeLists(mergedList,phraseDocs[getInd(reqInd)])
        else:
            resultList=mergedList+phraseDocs[getInd(reqInd)]
    else:
        mergedList=phraseDocs[0]+phraseDocs[1]
        resultList=mergedList+phraseDocs[2]
    return list(dict.fromkeys(resultList))


request='погода | лес | !спит'
request='погода & лес & !спит'

and_s='&'
or_s='|'
not_s='!'
folder=r'...\IS'
allDocs=['{0}'.format(i) for i in range(1,101)]


try:
    phrase=request.split()
    if len(phrase)<5 or phrase[1] not in [and_s,or_s] or phrase[3] not in [and_s,or_s]:
        raise Exception('Ошибка. Примеры:str1 & str2 | str3, str1 & !str2 | !str3, str1 | str2 | str3, str1 | !str2 | !str3 ')
    
    phrase=lemmatize(phrase)
    termDict=getTermDict()

    phraseDocs=getPhraseDocs(phrase,termDict)
    result=mergePhraseDocs(phrase,phraseDocs)
    result.sort(key=int)
    print('Документы удовлетворяют условию \'{0}\': {1}'.format(request,', '.join(result)))
except Exception as error:
    print(error)

Ошибка. Примеры:str1 & str2 | str3, str1 & !str2 | !str3, str1 | str2 | str3, str1 | !str2 | !str3 


# 4

1. Подсчитать tf каждого термина.
2. Подсчитать idf.
3. Подсчитать tf-idf.
В выводимых таблицах округлять значение до 3-5 знаков после запятой.

In [ ]:
from multiprocessing import Pool
import pandas as pd
import collections
import math

def termsIDF(folder,docsCount):
    indxf=open('{0}\windex.txt'.format(folder),"r",encoding='utf-8')
    lines=indxf.readlines()
    
    idf_terms={}
    for line in lines: 
        split=line.split()
        key=split[0]
        docs=split[1].split(',')
        idf_terms[key]=round(math.log10(docsCount/len(docs)),5)
        
    return idf_terms

def dfLine(i,term, tf, idf):
    return {'Document': i, 'Term': term, 'tf': tf, 'idf': idf, 'tf-idf': round(tf * idf,5)}

def docTermsTF(docName,terms):
    global lemmasFolder
    
    tf_terms={}
    f=open('{0}\{1}.txt'.format(lemmasFolder,docName), "r", encoding='utf-8')
    docTerms=f.readlines()
    docTerms=[term.split('\n')[0] for term in docTerms]
    termsCount = len(docTerms)
    termsCounter=collections.Counter(docTerms)
    for term in terms:
        tf_terms[term]=round(termsCounter[term]/termsCount,5)
    return tf_terms

def docDataFrame(i):
    global terms,idf_terms
    tf_terms=docTermsTF(i,terms)
    df=pd.DataFrame(columns=['Document','Term','tf','idf','tf-idf'])
    for term in tf_terms:
        tf=tf_terms[term]
        idf=idf_terms[term]
        df=df.append(dfLine(i,term,tf,idf),ignore_index=True)
    return df

folder = r'...\IS'
lemmasFolder=r'...\IS\Lemmas'

tdf=pd.DataFrame(columns=['Document','Term','tf','idf','tf-idf'])
idf_terms=termsIDF(folder,100)
indxf=open('{0}\windex.txt'.format(folder),"r",encoding='utf-8')
lines=indxf.readlines()
terms=[line.split()[0] for line in lines]


if __name__=='__main__':
    doc=[i for i in range(1,101)]
    with Pool(4) as pool:
        res=pool.map(docDataFrame,doc)
        tdf=tdf.append(res,ignore_index=True)
    try:
        tdf.to_excel('tdf.xlsx')
    except ValueError as error:
        part1=int(len(tdf)/2)
        tdf.head(part1).to_excel('{0}\\tdf1.xlsx'.format(folder))
        tdf.tail(len(tdf) - part1).to_excel('{0}\\tdf2.xlsx'.format(folder))
    print('finish!')

In [33]:
import pandas as pd
term=pd.read_excel('tdf1.xlsx',engine='openpyxl')
term.sample(9)

,Unnamed: 0,Document,Term,tf,idf,tf-idf
310591,310591,22,drm,0.0,2.00000,0.0
544002,544002,38,expertclubmicrosoft,0.0,1.69897,0.0
338427,338427,24,bwq5fc9wdj6h,0.0,1.69897,0.0
130006,130006,9,процесс,0.0,0.35655,0.0
410221,410221,29,addressing,0.0,2.00000,0.0
29570,29570,3,20reference,0.0,1.69897,0.0
535101,535101,37,ww-registration-azure-hybrid-,0.0,1.69897,0.0
155745,155745,11,together,0.0,0.92082,0.0
415103,415103,29,myxbox,0.0,2.00000,0.0


# Задание 5

1. Разработать поисковую систему на основе векторного поиска по построенному индексу.

Примеры запуска:
1. word1 
2. word1 word2
3. word1 word2 word3

In [75]:
import pandas as pd
import numpy as np
import pymorphy2

def lemmatize(words):
    morph=pymorphy2.MorphAnalyzer()
    terms=[morph.parse(word)[0].normal_form for word in words]
    return terms

def getRequestVector(lines,request):
    reqv=[0]*len(lines)
    for i in range(len(lines)):
        value=lines[i].split()[0]
        if value in request:
            reqv[i]=1
            request.remove(value)
    return reqv

def getCosangles(reqv,vectors):
    cosangles=[]
    for i in range(1,101):
        cosangle=np.dot(reqv,vectors[i]) / (np.linalg.norm(reqv)*np.linalg.norm(vectors[i]))
        cosangles.append(cosangle)
    return cosangles

def getDocuments(cosangles):
    docs={}
    for i in range(len(cosangles)):
        if cosangles[i] != 0:
            docs[i+1]=cosangles[i]
    docs={k: v for k, v in sorted(docs.items(),key=lambda item: item[1])}
    return docs

def getAllVectors(folder):
    df1=pd.read_excel(r'{0}\\tdf1.xlsx'.format(folder))
    df2=pd.read_excel(r'{0}\\tdf2.xlsx'.format(folder))
    df=df1.append(df2,ignore_index=True)
    vectors=df.groupby('Document')['tf-idf'].apply(list)
    return vectors

folder=r'...\IS'
indxf=open('{0}\windex.txt'.format(folder),'r',encoding='utf-8')
lines=indxf.readlines()
vectors=getAllVectors(folder)

request='покупать лицензии'
req=lemmatize(request.split())
reqv=getRequestVector(lines,req)

cosangles=getCosangles(reqv,vectors)
docs=getDocuments(cosangles)

indexf=open('{0}\index.txt'.format(folder), 'r', encoding='utf-8')
index=indexf.readlines()

documents=list(reversed(list(docs)))

print('Result:')
for i in documents:
    print(i,index[i-1].split()[1],docs[i])

Result:
79 https://www.microsoft.com/licensing/default 0.06240038222273482
22 https://www.microsoft.com/licensing/cloud-platform/windows-server 0.06239066382201604
58 https://www.microsoft.com/microsoft-365/fwlink/?LinkID=206977 0.0306067678616649
1 https://www.microsoft.com/microsoft-365/microsoft-office 0.030526784361693458
74 https://www.microsoft.com/ru-ru/store/b/business?icid=CNavBusinessStore/edge?form=MI13F3&OCID=MI13F3 0.02599644624122572
17 https://www.microsoft.com/ru-ru/store/b/business?icid=CNavBusinessStore 0.025926454745759706
95 https://www.microsoft.com/ru-ru/education/products/office/default.aspx 0.02169334961241924
38 https://www.microsoft.com/ru-ru/education/products/office/windows-10-apps 0.02169334961241924
100 https://www.microsoft.com/enterprise/government 0.012250731526532721
43 https://www.microsoft.com/enterprise 0.012245655379996879
71 https://www.microsoft.com/microsoft-365/business/windows?icid=CNavGamesWindowsGames 0.0043721359564300065
14 https://www.mic